<a href="https://colab.research.google.com/github/sanujsriv/Graph_Data_Mining_Project/blob/main/Graph_Data_Mining_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://linqs-data.soe.ucsc.edu/public/lbc/citeseer.tgz
!tar -xvzf citeseer.tgz
# !wget https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
# !tar -xvzf cora.tgz

--2021-05-15 03:08:18--  https://linqs-data.soe.ucsc.edu/public/lbc/citeseer.tgz
Resolving linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)... 128.114.47.74
Connecting to linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)|128.114.47.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359425 (351K) [application/x-gzip]
Saving to: ‘citeseer.tgz’

citeseer.tgz        100%[===================>] 351.00K  1.24MB/s    in 0.3s    

2021-05-15 03:08:18 (1.24 MB/s) - ‘citeseer.tgz’ saved [359425/359425]

citeseer/
citeseer/README
citeseer/citeseer.cites
citeseer/citeseer.content


# Start

In [ ]:
!nvidia-smi

In [ ]:
# !git clone https://github.com/maxiaoba/GRAPE.git
# !wget https://raw.githubusercontent.com/maxiaoba/GRAPE/master/uci/raw_data/concrete/data/data.txt
!wget https://raw.githubusercontent.com/sanujsriv/datasets/main/reuters_data_labels.zip

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-geometric
!pip install fancyimpute -U

In [ ]:
# import torch
# import torch_sparse
# torch.version.cuda,torch.__version__,torch_sparse.__version__

In [ ]:
!unzip ./reuters_data_labels.zip

Archive:  ./reuters_data_labels.zip
  inflating: retuers_data.pkl        
  inflating: retuers_labels.pkl      


In [ ]:
#@title function : load / save pickle_obj
import pickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Data Loading


In [ ]:
from sklearn import preprocessing
import torch
import pandas as pd
import numpy as np
# from torch_geometric.data import Data
import os.path as osp
import inspect
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as opt
from torch.nn import Parameter

In [ ]:
def save_mask(length,true_rate,log_dir,seed):
    np.random.seed(seed)
    mask = np.random.rand(length) < true_rate
    np.save(osp.join(log_dir,'len'+str(length)+'rate'+str(true_rate)+'seed'+str(seed)),mask)
    return mask

def mask_edge(edge_index,edge_attr,mask,remove_edge):
    edge_index = edge_index.clone().detach()
    edge_attr = edge_attr.clone().detach()
    if remove_edge:
        edge_index = edge_index[:,mask]
        edge_attr = edge_attr[mask]
    else:
        edge_attr[~mask] = 0.
    return edge_index, edge_attr
    
def get_known_mask(known_prob, edge_num):
    known_mask = (torch.FloatTensor(edge_num, 1).uniform_() < known_prob).view(-1)
    return known_mask

In [ ]:
def create_node(df): # # onehot feature node, sampling of single node
        nrow, ncol = df.shape
        feature_ind = np.array(range(ncol))
        feature_node = np.zeros((ncol,ncol))
        feature_node[np.arange(ncol), feature_ind] = 1
        sample_node = [[1]*ncol for i in range(nrow)]
        node = sample_node + feature_node.tolist()
        return node

def create_edge_attr(df):
    nrow, ncol = df.shape
    edge_attr = []
    for i in range(nrow):
        for j in range(ncol):
            edge_attr.append([float(df.iloc[i,j])])
    edge_attr = edge_attr + edge_attr
    return edge_attr

def create_edge(df):
    n_row, n_col = df.shape
    edge_start = []
    edge_end = []
    for x in range(n_row):
        edge_start = edge_start + [x] * n_col # obj
        edge_end = edge_end + list(n_row+np.arange(n_col)) # att
        if(x==n_row-1):
          print(edge_start)    
    edge_start_new = edge_start + edge_end
    edge_end_new = edge_end + edge_start
    return (edge_start_new, edge_end_new)

In [ ]:
def get_data(df_X, df_y, train_edge_prob=0.7, split_sample_ratio=0., split_by='y', train_y_prob=0.7, seed=0, normalize=True):
    
    if len(df_y.shape)==1:
        df_y = df_y.to_numpy()
    elif len(df_y.shape)==2:
        df_y = df_y[0].to_numpy()

    if normalize:
        x = df_X.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_X = pd.DataFrame(x_scaled)
        
    edge_start, edge_end = create_edge(df_X)
    edge_index = torch.tensor([edge_start, edge_end], dtype=int)
    edge_attr = torch.tensor(create_edge_attr(df_X), dtype=torch.float)
    node_init = create_node(df_X) 

    x = torch.tensor(node_init, dtype=torch.float)
    y = torch.tensor(df_y, dtype=torch.float)

    torch.manual_seed(seed)

    train_edge_mask = get_known_mask(train_edge_prob, int(edge_attr.shape[0]/2))
    double_train_edge_mask = torch.cat((train_edge_mask, train_edge_mask), dim=0)

    #mask edges based on the generated train_edge_mask
    #train_edge_index is known, test_edge_index in unknwon, i.e. missing
    train_edge_index, train_edge_attr = mask_edge(edge_index, edge_attr,
                                                double_train_edge_mask, True)
    train_labels = train_edge_attr[:int(train_edge_attr.shape[0]/2),0]
    test_edge_index, test_edge_attr = mask_edge(edge_index, edge_attr,
                                                ~double_train_edge_mask, True)
    test_labels = test_edge_attr[:int(test_edge_attr.shape[0]/2),0]
    #mask the y-values during training, i.e. how we split the training and test sets
    train_y_mask = get_known_mask(train_y_prob, y.shape[0])
    test_y_mask = ~train_y_mask

    data = Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr,
            train_y_mask=train_y_mask, test_y_mask=test_y_mask,
            train_edge_index=train_edge_index,train_edge_attr=train_edge_attr,
            train_edge_mask=train_edge_mask,train_labels=train_labels,
            test_edge_index=test_edge_index,test_edge_attr=test_edge_attr,
            test_edge_mask=~train_edge_mask,test_labels=test_labels, 
            df_X=df_X,df_y=df_y,
            edge_attr_dim=train_edge_attr.shape[-1],
            user_num=df_X.shape[0]
            )
    return data

In [ ]:
def get_df(name):
  name = name.lower()
  if (name == 'concrete'):
    path = osp.dirname(osp.abspath(inspect.getfile(inspect.currentframe())))
    df_np = np.loadtxt(path+'/data.txt') # concrete UCI data
    df_concrete = pd.DataFrame(df_np,columns=['Cement','Blast Furnace Slag','Fly Ash ','Water','Superplasticizer','Coarse Aggregate ','Fine Aggregate ','Age','Concrete compressive strength'])
    return df_concrete
  elif (name == 'cora'): 
    feature_names = ["w_{}".format(ii) for ii in range(1433)]
    column_names =  feature_names + ["subject"]
    node_data = pd.read_csv(os.path.join('/content/cora', "cora.content"), sep='\t', header=None, names=column_names)
    edgelist = pd.read_csv(os.path.join('/content/cora', "cora.cites"), sep='\t', header=None, names=["target", "source"])
    edgelist["label"] = "cites"
    return node_data
  elif (name == 'citeseer'): 
    feature_names = ["w_{}".format(ii) for ii in range(3703)]
    column_names =  feature_names + ["subject"]
    node_data = pd.read_csv(os.path.join('/content/citeseer', "citeseer.content"), sep='\t', header=None, names=column_names)
    edgelist = pd.read_csv(os.path.join('/content/citeseer', "citeseer.cites"), sep='\t', header=None, names=["target", "source"])
    edgelist["label"] = "cites"
    return node_data
  elif (name == 'reuters'):
    data_preprocessed = load_obj('retuers_data')
    data_preprocessed_labels = load_obj('retuers_labels')
    preprossed_data = data_preprocessed
    train_label = data_preprocessed_labels
    vectorizer = CountVectorizer(min_df=0.01,max_df=0.4)
    train_vec = vectorizer.fit_transform(preprossed_data).toarray()
    vocab = vectorizer.vocabulary_
    id_vocab = dict(map(reversed, vocab.items()))
    nonzeros_indexes = np.where(train_vec.any(1))[0]
    train_vec_non_zeros = [train_vec[i] for i in nonzeros_indexes]
    preprossed_data_non_zeros = [preprossed_data[i] for i in nonzeros_indexes]
    train_label = [data_preprocessed_labels[i] for i in nonzeros_indexes]
    train_vec = np.asanyarray(train_vec_non_zeros)
    sorted_id_word_vocab = sorted(id_vocab.items(), key=lambda x: x[1]) ### alphabetically sorted
    word_list = [s[1] for s in sorted_id_word_vocab]
    df = pd.DataFrame(train_vec,columns=word_list)
    df['label']=train_label
    return df


In [ ]:
df_data = 'citeseer'
df = get_df(df_data)
df

['w_0', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12', 'w_13', 'w_14', 'w_15', 'w_16', 'w_17', 'w_18', 'w_19', 'w_20', 'w_21', 'w_22', 'w_23', 'w_24', 'w_25', 'w_26', 'w_27', 'w_28', 'w_29', 'w_30', 'w_31', 'w_32', 'w_33', 'w_34', 'w_35', 'w_36', 'w_37', 'w_38', 'w_39', 'w_40', 'w_41', 'w_42', 'w_43', 'w_44', 'w_45', 'w_46', 'w_47', 'w_48', 'w_49', 'w_50', 'w_51', 'w_52', 'w_53', 'w_54', 'w_55', 'w_56', 'w_57', 'w_58', 'w_59', 'w_60', 'w_61', 'w_62', 'w_63', 'w_64', 'w_65', 'w_66', 'w_67', 'w_68', 'w_69', 'w_70', 'w_71', 'w_72', 'w_73', 'w_74', 'w_75', 'w_76', 'w_77', 'w_78', 'w_79', 'w_80', 'w_81', 'w_82', 'w_83', 'w_84', 'w_85', 'w_86', 'w_87', 'w_88', 'w_89', 'w_90', 'w_91', 'w_92', 'w_93', 'w_94', 'w_95', 'w_96', 'w_97', 'w_98', 'w_99', 'w_100', 'w_101', 'w_102', 'w_103', 'w_104', 'w_105', 'w_106', 'w_107', 'w_108', 'w_109', 'w_110', 'w_111', 'w_112', 'w_113', 'w_114', 'w_115', 'w_116', 'w_117', 'w_118', 'w_119', 'w_120', 'w_121', 'w_122', 'w

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,...,w_3664,w_3665,w_3666,w_3667,w_3668,w_3669,w_3670,w_3671,w_3672,w_3673,w_3674,w_3675,w_3676,w_3677,w_3678,w_3679,w_3680,w_3681,w_3682,w_3683,w_3684,w_3685,w_3686,w_3687,w_3688,w_3689,w_3690,w_3691,w_3692,w_3693,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,subject
100157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agents
100598,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,IR
105684,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agents
11099,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DB
114091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zhang99query,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,DB
zhang99situated,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,ML
zhang99towards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,IR
zhou00implementation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,DB


In [ ]:
if df_data =='cora' or 'citeseer':
  df['subject'],_ = pd.factorize(df['subject'])
if df_data =='reuters':
  df['label'],_ = pd.factorize(df['label'])
df_np = df.to_numpy()
df_X = pd.DataFrame(df_np[:, :-1])
df_y = pd.DataFrame(df_np[:, -1:])
# data = get_data(df_X,df_y)
log_path = './result/'

In [ ]:
np.savetxt(fname='citeseer.txt',X=df_np[:50],delimiter='  ',fmt='%1.1f')

In [ ]:
# np.savetxt(fname='cora.txt',X=df_np[:df_np.shape[0]//20],delimiter='  ',fmt='%1.1f')

In [ ]:
np.savetxt(fname='reuters.txt',X=df_np[:500],delimiter='  ',fmt='%1.1f')

In [ ]:
filename = df_data+'_GRAPE'
save_obj(data,filename)

In [ ]:
!zip ./cora_GRAPE.zip ./cora_GRAPE.pkl

  adding: cora_GRAPE.pkl (deflated 97%)


# Building the Model (*In Progress....*)


In [ ]:
def get_activation(activation):
    if activation == 'relu':
        return torch.nn.ReLU()
    elif activation == 'prelu':
        return torch.nn.PReLU()
    elif activation == 'tanh':
        return torch.nn.Tanh()
    elif (activation is None) or (activation == 'none'):
        return torch.nn.Identity()
    else:
        raise NotImplementedError

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

class GNNTextData(torch.nn.Module):
    def __init__(self, 
                node_input_dim, edge_input_dim,
                node_dim, edge_dim, edge_mode,
                model_types, dropout, activation,
                concat_states, node_post_mlp_hiddens,
                normalize_embs, aggr
                ):
        super(GNNTextData, self).__init__()
        # self.dropout = dropout
        # self.activation = activation
        self.concat_states = concat_states
        self.model_types = model_types
        self.gnn_layer_num = len(model_types)

        # convs
        self.convs = self.build_convs(node_input_dim, edge_input_dim,
                                    node_dim, edge_dim, edge_mode,
                                    model_types, normalize_embs, activation, aggr)

        # post node update
        if concat_states:
            self.node_post_mlp = self.build_node_post_mlp(int(node_dim*len(model_types)), int(node_dim*len(model_types)), node_post_mlp_hiddens, dropout, activation)
        else:
            self.node_post_mlp = self.build_node_post_mlp(node_dim, node_dim, node_post_mlp_hiddens, dropout, activation)

        self.edge_update_mlps = self.build_edge_update_mlps(node_dim, edge_input_dim, edge_dim, self.gnn_layer_num, activation)

    def build_node_post_mlp(self, input_dim, output_dim, hidden_dims, dropout, activation):
        if 0 in hidden_dims:
            return get_activation('none')
        else:
            layers = []
            for hidden_dim in hidden_dims:
                layer = nn.Sequential(
                            nn.Linear(input_dim, hidden_dim),
                            get_activation(activation),
                            nn.Dropout(dropout),
                            )
                layers.append(layer)
                input_dim = hidden_dim
            layer = nn.Linear(input_dim, output_dim)
            layers.append(layer)
            return nn.Sequential(*layers)

    def build_convs(self, node_input_dim, edge_input_dim,
                     node_dim, edge_dim, edge_mode,
                     model_types, normalize_embs, activation, aggr):
        convs = nn.ModuleList()
        conv = self.build_conv_model(model_types[0],node_input_dim,node_dim,
                                    edge_input_dim, edge_mode, normalize_embs[0], activation, aggr)
        convs.append(conv)
        for l in range(1,len(model_types)):
            conv = self.build_conv_model(model_types[l],node_dim, node_dim,
                                    edge_dim, edge_mode, normalize_embs[l], activation, aggr)
            convs.append(conv)
        return convs

    def build_conv_model(self, model_type, node_in_dim, node_out_dim, edge_dim, edge_mode, normalize_emb, activation, aggr):
        #print(model_type)
        if model_type == 'GCN':
            return pyg_nn.GCNConv(node_in_dim,node_out_dim)
        elif model_type == 'GraphSage':
            return pyg_nn.SAGEConv(node_in_dim,node_out_dim)
        elif model_type == 'GAT':
            return pyg_nn.GATConv(node_in_dim,node_out_dim)


    def build_edge_update_mlps(self, node_dim, edge_input_dim, edge_dim, gnn_layer_num, activation):
        edge_update_mlps = nn.ModuleList()
        edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_input_dim,edge_dim),
                get_activation(activation),
                )
        edge_update_mlps.append(edge_update_mlp)
        for l in range(1,gnn_layer_num):
            edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_dim,edge_dim),
                get_activation(activation),
                )
            edge_update_mlps.append(edge_update_mlp)
        return edge_update_mlps

    def update_edge_attr(self, x, edge_attr, edge_index, mlp):
        x_i = x[edge_index[0],:]
        x_j = x[edge_index[1],:]
        edge_attr = mlp(torch.cat((x_i,x_j,edge_attr),dim=-1))
        return edge_attr

    def forward(self, x, edge_attr, edge_index):
        if self.concat_states:
            concat_x = []
        for l,(conv_name,conv) in enumerate(zip(self.model_types,self.convs)):
            # self.check_input(x,edge_attr,edge_index)
            if conv_name == 'EGCN' or conv_name == 'EGSAGE':
                x = conv(x, edge_attr, edge_index)
            else:
                x = conv(x, edge_index)
            if self.concat_states:
                concat_x.append(x)
            edge_attr = self.update_edge_attr(x, edge_attr, edge_index, self.edge_update_mlps[l])
            #print(edge_attr.shape)
        if self.concat_states:
            x = torch.cat(concat_x, 1)
        x = self.node_post_mlp(x)
        # self.check_input(x,edge_attr,edge_index)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class MLPNet(torch.nn.Module):
    def __init__(self, 
         		input_dims, output_dim,
         		hidden_layer_sizes=(64,),
         		hidden_activation='relu',
         		output_activation=None,
                dropout=0.):
        super(MLPNet, self).__init__()

        layers = nn.ModuleList()

        input_dim = np.sum(input_dims)

        for layer_size in hidden_layer_sizes:
        	hidden_dim = layer_size
        	layer = nn.Sequential(
        				nn.Linear(input_dim, hidden_dim),
        				get_activation(hidden_activation),
        				nn.Dropout(dropout),
        				)
        	layers.append(layer)
        	input_dim = hidden_dim

        layer = nn.Sequential(
        				nn.Linear(input_dim, output_dim),
        				get_activation(output_activation),
        				)
       	layers.append(layer)
       	self.layers = layers

    def forward(self, inputs):
    	if torch.is_tensor(inputs):
    		inputs = [inputs]
    	input_var = torch.cat(inputs,-1)
    	for layer in self.layers:
    		input_var = layer(input_var)
    	return input_var

In [ ]:
if torch.cuda.is_available():
  cuda = auto_select_gpu()
  device = torch.device('cuda:{}'.format(cuda))
else:
  device = torch.device('cpu')

model = GNNTextData()
impute_model = MLPNet(input_dims=128, output_dim=1).to(device)

trainable_parameters = list(impute_model.parameters())
print("total trainable_parameters: ",len(trainable_parameters))

# train
Train_loss = []
Test_rmse = []
Test_l1 = []
Lr = []

x = data.x.clone().detach().to(device)
all_train_edge_index = data.train_edge_index.clone().detach().to(device)
all_train_edge_attr = data.train_edge_attr.clone().detach().to(device)
all_train_labels = data.train_labels.clone().detach().to(device)
test_input_edge_index = all_train_edge_index
test_input_edge_attr = all_train_edge_attr
test_edge_index = data.test_edge_index.clone().detach().to(device)
test_edge_attr = data.test_edge_attr.clone().detach().to(device)
test_labels = data.test_labels.clone().detach().to(device)

train_edge_index, train_edge_attr, train_labels =\
             all_train_edge_index, all_train_edge_index, all_train_labels
print("train edge num is {}, test edge num is input {}, output {}"\
                .format(
                all_train_edge_index.shape[0],
                test_input_edge_attr.shape[0], test_edge_attr.shape[0]))

learning_rate = 0.00002
optimizer = opt.Adam(impute_model.parameters(), learning_rate)

for epoch in range(1000):
   
  #  impute_model.train()
   optimizer.zero_grad()
   pred = impute_model([x_embd[train_edge_index[0]], x_embd[train_edge_index[1]]])
   loss.backward()
   optimizer.step()
   loss = F.mse_loss(pred_train, label_train)

In [ ]:
impute_model.eval()
with torch.no_grad():
  pred = impute_model([x_embd[valid_edge_index[0], :], x_embd[valid_edge_index[1], :]])

In [ ]:
# pred_train = pred_train.detach().cpu().numpy()
# label_train = label_train.detach().cpu().numpy()
# pred_test = pred_test.detach().cpu().numpy()
# label_test = label_test.detach().cpu().numpy()

# **GRAPE Result & Other Baselines**


In [ ]:
#@title function : load / save pickle_obj
import pickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
import os
from utils.plot_utils import plot_curve, plot_sample
log_path = './results_all/'

In [ ]:
model_result = load_obj('uci/test/concrete/0/result')

def get_load_obj(baseline_name):
  return load_obj('/content/GRAPE/uci/mdi_results/results/'+baseline_name+'/concrete/0/result')

obj = model_result
names = []

plot_curve(obj['curves'], log_path+'curves.png',keys=None, 
                clip=True, label_min=True, label_end=True)
plot_curve(obj, log_path+'lr.png',keys=['lr'], 
            clip=False, label_min=False, label_end=False)
plot_sample(obj['outputs'], log_path+'outputs.png', 
            groups=[['final_pred_train','label_train'],
                        ['final_pred_test','label_test']
                        ], 
                num_points=20)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plotCurve(data,keys=None,clip=True, label_min=True, label_end=True):
    if not keys:
        keys = data.keys()
    plt.figure(figsize=(10,10))
    for i,key in enumerate(keys):
        if key == 'train_loss':
          continue
        plt.subplot(len(keys),1,i+1)
        if clip:
            limit = 2*np.mean(np.abs(data[key]))
            y = np.clip(data[key],-limit,limit)
        else:
            y = data[key]
        plt.plot(y, linewidth=1.,label=key)
        if label_min:
            plt.plot(np.argmin(data[key]),np.min(data[key]),'o',
                    label="min: {:.3g}".format(np.min(data[key])))
        if label_end:
            plt.plot(len(data[key])-1,data[key][-1],'o',
                    label="end: {:.3g}".format(data[key][-1]))
        plt.legend()        

In [ ]:
baseline_models= ['knn_v1','mean_v1','svd_v1','spectral_v1']
for b in baseline_models:
  baseline = get_load_obj(b)
  print(b," (rmse) : ",baseline['rmse'])
  print(b," (mae) : ",baseline['mae'])
  print()

In [ ]:
plotCurve(obj['curves'],keys=None,clip=True, label_min=True, label_end=True)

In [ ]:
!zip -r ./results_all.zip ./results_all/
!zip -r ./baseline_results_all.zip ./uci/mdi_results/results/
!zip -r ./model_results.zip  ./uci/test/concrete

In [ ]:
from google.colab import files
files.download("./results_all.zip")
files.download("./baseline_results_all.zip")
files.download("./model_results.zip")